In [1]:
path_data = '/data'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
import os

import pandas as pd

import torch
from pkg_resources import packaging
from PIL import Image
import tqdm
from pathlib import Path
print("Torch version:", torch.__version__)

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

plt.style.use('ggplot')

Torch version: 2.2.1+cu121


In [3]:
data_unfiltered = pd.read_csv("/".join([path_data, "images_v2.csv"]))
#display(data_unfiltered)

In [4]:
len(data_unfiltered)

38479

### Remove subjective topics

In [5]:
subjective_topics = ['Favorite home decorations', 'Favourite item in kitchen', 'Favourite sports clubs', 'How the most loved item is used', 'icons', 'Idols', 'Latest furniture bought', ' looking over the shoulder', 'Most loved item', 'Most loved toy', 'Most played songs on the radio', 'Music idol', 'Next big thing you are planning to buy', 'Playing with most loved toy', 'Thing I dream about having', 'Things I wish I had', 'Using most loved item', 'Youth culture', 'What I wish I could buy']

In [6]:
unique_topics = data_unfiltered['topics'].unique()

In [7]:
Subjective_topic_list = []

for topic in unique_topics:
    topic_ = topic.replace('/ ', ', ')
    topic_split = topic_.split(',')
    for split in topic_split:
        if split in subjective_topics:
            Subjective_topic_list.append(topic)

In [8]:
data = data_unfiltered[~data_unfiltered['topics'].isin(Subjective_topic_list)]
display(data)

,id,country.name,country.id,region.id,type,imageRelPath,topics,place,income
1,5d4bf31ccf0b3a0f3f359814,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359814/5d4bf31ccf0b3a...,Family snapshots,butoyi,26.994581
2,5d4bf31ccf0b3a0f3f35982a,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982a/5d4bf31ccf0b3a...,Cutlery,butoyi,26.994581
3,5d4bf31ccf0b3a0f3f35982e,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982e/5d4bf31ccf0b3a...,Family,butoyi,26.994581
4,5d4bf31ccf0b3a0f3f359830,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359830/5d4bf31ccf0b3a...,Place where eating dinner,butoyi,26.994581
5,5d4bf31dcf0b3a0f3f35983c,Burundi,bi,af,image,assets/5d4bf31dcf0b3a0f3f35983c/5d4bf31dcf0b3a...,Plate of food,butoyi,26.994581
...,...,...,...,...,...,...,...,...,...
38474,5ec4f5513f62767d97a47324,France,fr,eu,image,assets/5ec4f5513f62767d97a47324/5ec4f5513f6276...,Bed,larriere,19671.000000
38475,5ec4f5513f62767d97a47325,France,fr,eu,image,assets/5ec4f5513f62767d97a47325/5ec4f5513f6276...,Bathroom/Toilet,larriere,19671.000000
38476,5ec4f5523f62767d97a47327,France,fr,eu,image,assets/5ec4f5523f62767d97a47327/5ec4f5523f6276...,Armchair,larriere,19671.000000
38477,5ec4f5523f62767d97a47328,France,fr,eu,image,assets/5ec4f5523f62767d97a47328/5ec4f5523f6276...,Armchair,larriere,19671.000000


In [9]:
len(data)

36754

### Split topics

In [10]:
from collections import Counter
list_topics = list(data['topics'])


separate_topics = [t.lower().strip() for topic in list_topics for t in topic.split(",") ]


print(len(separate_topics))
#print(Counter(separate_topics))
set_topics = list(set(separate_topics))
print(len(set_topics))

45692
270


In [11]:
# map each topic to list of corresponding images
dict_topic2img = {}
for list_topics, image_path in zip(data['topics'], data['imageRelPath']):
    for topic in list_topics.split(","):
        topic = topic.lower().strip()
        if topic not in dict_topic2img:
            dict_topic2img[topic] = set() #### here a set was used in place of list to avoid duplicate where keyword appears twice in a topic
        dict_topic2img[topic].add(image_path)

print(len(dict_topic2img))

270


In [12]:
#sanity checks 

ground_truth_counts = [len(dict_topic2img[i]) for i in dict_topic2img]
print(sum(ground_truth_counts))

45616


In [13]:
topics = dict_topic2img.keys()

# Create prompts for the text encoder

In [14]:
temp = "This is a photo of "
English_topics = [temp + topic for topic in topics]

In [15]:
#English_topics

In [16]:
topic2english = {}
for topic, prompt in zip(topics, English_topics):
    topic2english[topic] = prompt

In [17]:
topic2english['family snapshots']

'This is a photo of family snapshots'

In [18]:
english_dict_topic2imgGT = {}

for topic in topics:
    key = topic2english[topic]
    english_dict_topic2imgGT[key] = dict_topic2img[topic]
print(len(english_dict_topic2imgGT))

270


### Make English-country prompts

In [19]:
income_grps = ['a poor country', 'a low income country', 'a cheap country', 'an impoverished country', 'a rich country', 'a high income country', 'an expensive country', 'a wealthy country', 'a country', 'a poor region', 'a low income region', 'a cheap region', 'an impoverished region', 'a rich region', ' a high income region', 'an expensive region', 'a wealthy region', 'a region', 'a poor place', 'a low income place', 'a cheap place', 'an impoverished place', 'a rich place', 'a high income place', 'an expensive place', 'a wealthy place', 'a place', 'a poor home',  'a low income home', 'a cheap home', 'an impoverished home', 'a rich home', 'a high income home', 'an expensive home', 'a wealthy home', 'a home']

In [20]:
len(income_grps)

36

In [21]:
list_of_income_dicts = []  # list of ground truth imgs and text prompt mapping for each country; list of dict_topic2img for each country
list_of_topic2prompt_dict = []  # list of text prompt and topic mapping

for income in income_grps:
    
    income_topics = ['This is a photo of ' + topic + ' from ' + income for topic in topics]
      
    topic2income = {}
    for topic, prompt in zip(topics, income_topics):
        topic2income[topic] = prompt
    list_of_topic2prompt_dict.append(topic2income)
    
    
    dict_incometopic2imgGT = {}

    for topic in topics:
        key = topic2income[topic]
        dict_incometopic2imgGT[key] = dict_topic2img[topic]
        
    list_of_income_dicts.append(dict_incometopic2imgGT)

In [22]:
len(list_of_income_dicts)

36

In [24]:
len(list_of_income_dicts[1].keys())

270

### Load img embeddings

In [31]:
import pickle
with open("nllb_img_embedding.pkl", 'rb') as f:
    img_embedding = pickle.load(f)

In [32]:
img_embedding.shape

torch.Size([36754, 1152])

## Cosine sim for country_prompts and imgs

In [33]:
from sentence_transformers import SentenceTransformer, util
from open_clip import get_tokenizer

from open_clip import create_model_from_pretrained
from PIL import Image, ImageFile
import timm

In [34]:
# Load model directly
from transformers import AutoProcessor, AutoModel

model, transform = create_model_from_pretrained("nllb-clip-large-siglip", "v1", device="cuda")
model = model.eval()


In [35]:
from open_clip import get_tokenizer

tokenizer = get_tokenizer("nllb-clip-large-siglip")

In [36]:
list_income_score_dicts = []
for income_prompt in tqdm.tqdm(list_of_topic2prompt_dict):
    text = list(income_prompt.values())
    text_langs = 'eng_Latn'
    text_inputs = []
    for i in range(len(text)):
        tokenizer.set_language(text_langs)
        text_inputs.append(tokenizer(text[i]))
        
    text_inputs = torch.stack(text_inputs).squeeze(1)
    text_inputs = text_inputs.to("cuda")
    
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)

    cossim = util.cos_sim(text_features, img_embedding)
    cos_sim = cossim.cpu().numpy()

    dict_topic2score = {}
    for i, topic in enumerate(income_prompt.values()):
        clip_scores = cos_sim[i] # clip scores between topic and images from data['imageRelPath']

        list_scores_img = list(zip(clip_scores, data['imageRelPath'], data['topics'])) # ******
        dict_topic2score[topic] = sorted(list_scores_img, key = lambda x: x[0], reverse=True)

    list_income_score_dicts.append(dict_topic2score)

100%|██████████| 36/36 [05:39<00:00,  9.44s/it]


In [37]:
len(list_income_score_dicts)

36

In [38]:
len(list_income_score_dicts[0])

270

In [ ]:
#list_country_score_dicts[0]['This is a photo of family snapshots from Peru'][1]

In [ ]:
# This runs FOREVER and takes so much space... BEWARE!

# f = open("list_country_dicts.pkl","wb")

# # write the python object (dict) to pickle file
# pickle.dump(list_country_score_dicts,f)

# # close file
# f.close()

In [ ]:
# import pickle
# with open("list_country_dicts.pkl", 'rb') as f:
#     list_country_dicts = pickle.load(f)

### Score Processing

In [39]:
labels=["poor", "lower-middle", "upper-middle", "rich"]

#list_of_topic2prompt_dict[1]
data["quartile"] = pd.qcut(data["income"], q=[0, 0.25, 0.5, 0.75, 1],    
                         labels=labels)

In [40]:
# map each image to corresponding topics, countries, income level ... TODO
dict_img2topic, dict_img2country, dict_img2incomelevel, dict_country2region = {}, {}, {}, {}
for list_topics, image_path, country, incomelevel, region in zip(data['topics'], data['imageRelPath'], data['country.name'], data['quartile'], data['region.id']):
    dict_img2topic[image_path] = list_topics
    dict_img2country[image_path] = country
    dict_img2incomelevel[image_path] = incomelevel
    dict_country2region[country] = region

print(len(dict_img2topic))
# print(dict_img2country)

36754


In [37]:
## Trial for the first country_dict  (recall and precision and F1 scores are equal)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

all_images = list(data['imageRelPath'])
list_dict_topic2FNimg = []
list_dict_topic2F1 = []

for income_dict in tqdm.tqdm(list_income_score_dicts[:1]):
    
    dict_topic2FNimg = {}
    dict_topic2F1 = {}
    
    for topic in tqdm.tqdm(list(income_dict.keys())):
        dict_income_topic2img = next(item for item in list_of_income_dicts if topic in item.keys())
        count_pos = len(dict_income_topic2img[topic])
        
        GT_pos_images = dict_income_topic2img[topic]
        Pred_pos_images = [data[1] for data in income_dict[topic][:count_pos]]

        GT_labels = [1 if img in GT_pos_images else 0 for img in all_images]
        Pred_labels = [1 if img in Pred_pos_images else 0 for img in all_images]
        
        
        accuracy = round(accuracy_score(GT_labels, Pred_labels) * 100, 2)
        precision = round(precision_score(GT_labels, Pred_labels) * 100, 2)
        recall = round(recall_score(GT_labels, Pred_labels) * 100, 2)
        f1 = round(f1_score(GT_labels, Pred_labels) * 100, 2)
        
        print(f"Acc: {accuracy}, Recall: {recall}, Precision: {precision}, F1: {f1}")
        
        dict_topic2F1[topic] = f1 
        list_dict_topic2F1.append(dict_topic2F1)
        
        
        # What are the forgotten items (FN) 
        images_FN = []
        for idx, img in enumerate(all_images):
            label_GT = GT_labels[idx]
            label_Pred = Pred_labels[idx]
            #FN
            if label_GT == 1 and label_Pred == 0:
                images_FN.append(img) 
        
        dict_topic2FNimg[topic] = images_FN
        
        list_dict_topic2FNimg.append(dict_topic2FNimg)
        

  0%|          | 1/270 [00:00<00:54,  4.90it/s]

Acc: 99.48, Recall: 5.88, Precision: 5.88, F1: 5.88



  1%|          | 2/270 [00:00<01:11,  3.74it/s]

Acc: 98.54, Recall: 10.63, Precision: 10.63, F1: 10.63



  1%|          | 3/270 [00:00<01:35,  2.78it/s]

...

Acc: 99.99, Recall: 0.0, Precision: 0.0, F1: 0.0
Acc: 99.99, Recall: 0.0, Precision: 0.0, F1: 0.0



 99%|█████████▉| 268/270 [01:04<00:00,  6.80it/s]

Acc: 99.99, Recall: 0.0, Precision: 0.0, F1: 0.0
Acc: 99.99, Recall: 0.0, Precision: 0.0, F1: 0.0



100%|██████████| 1/1 [01:05<00:00, 65.12s/it]

Acc: 99.99, Recall: 0.0, Precision: 0.0, F1: 0.0
Acc: 99.99, Recall: 0.0, Precision: 0.0, F1: 0.0


In [74]:
#dict_topic2F1

In [ ]:
#len(dict_topic2FNimg['This is a photo of family snapshots from Peru'])

In [41]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

all_images = list(data['imageRelPath'])
list_dict_topic2FNimg = []
list_dict_topic2F1 = []

for income_dict in tqdm.tqdm(list_income_score_dicts):
    
    dict_topic2FNimg = {}
    dict_topic2F1 = {}
    
    for topic in list(income_dict.keys()):
        dict_income_topic2img = next(item for item in list_of_income_dicts if topic in item.keys())
        count_pos = len(dict_income_topic2img[topic])
        
        GT_pos_images = dict_income_topic2img[topic]
        Pred_pos_images = [data[1] for data in income_dict[topic][:count_pos]]

        GT_labels = [1 if img in GT_pos_images else 0 for img in all_images]
        Pred_labels = [1 if img in Pred_pos_images else 0 for img in all_images]
        
        
        accuracy = round(accuracy_score(GT_labels, Pred_labels) * 100, 2)
        precision = round(precision_score(GT_labels, Pred_labels) * 100, 2)
        recall = round(recall_score(GT_labels, Pred_labels) * 100, 2)
        f1 = round(f1_score(GT_labels, Pred_labels) * 100, 2)
        
        #print(f"Acc: {accuracy}, Recall: {recall}, Precision: {precision}, F1: {f1}")
        
        dict_topic2F1[topic] = f1
        
        
        
        # What are the forgotten items (FN) 
        images_FN = []
        for idx, img in enumerate(all_images):
            label_GT = GT_labels[idx]
            label_Pred = Pred_labels[idx]
            #FN
            if label_GT == 1 and label_Pred == 0:
                images_FN.append(img) 
        
        dict_topic2FNimg[topic] = images_FN
        
    list_dict_topic2FNimg.append(dict_topic2FNimg)
    list_dict_topic2F1.append(dict_topic2F1)

100%|██████████| 36/36 [38:43<00:00, 64.53s/it]


In [44]:
len(list_dict_topic2F1[0])

270

In [45]:
list_dict_topic2F1[1]

{'This is a photo of family snapshots from a low income country': 2.94,
 'This is a photo of cutlery from a low income country': 4.32,
 'This is a photo of family from a low income country': 52.7,
 'This is a photo of place where eating dinner from a low income country': 13.23,
 'This is a photo of plate of food from a low income country': 48.23,
 'This is a photo of table with food from a low income country': 41.7,
 'This is a photo of get water from a low income country': 23.71,
 'This is a photo of stove/hob from a low income country': 41.4,
 'This is a photo of light source in kitchen from a low income country': 32.36,
 'This is a photo of drinking water from a low income country': 18.89,
 'This is a photo of kitchen sink from a low income country': 37.5,
 'This is a photo of bathroom privacy from a low income country': 17.34,
 'This is a photo of backyard from a low income country': 22.67,
 'This is a photo of front door from a low income country': 14.11,
 'This is a photo of home

In [42]:
f = open("data/income_list_dict_topic2FNimg.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(list_dict_topic2FNimg,f)

# close file
f.close()

In [43]:
f = open("data/income_list_dict_topic2F1.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(list_dict_topic2F1,f)

# close file
f.close()